# CSE 255 Programming Assignment 5

##  Problem Statement

In this programming assignment, you will estimate intrinsic dimension by calculating the Mean Squared Distance of the entire dataset to their representative centers. You will use the K-Means API in spark to find representative centers.
All of the necessary helper code is included in this notebook. However, we advise you to go over the lecture material, the EdX videos and the corresponding notebooks before you attempt this Programming Assignment.

## Reviewing the Theory 

### Computing the intrinsic dimension of a data set
Recall from class that given any $d$ dimensional dataset, we can divide it into $n$ cells of diameter $\epsilon$ each. The relationship between $n, d, \epsilon$ is then given by:
$$
n = \frac{C}{\epsilon^d}
$$
Where $C \in I\!R$

Alternately, we may write this as:
$$
\log{n} = \log{C} + d \times \log{\frac{1}{\epsilon}}
$$

Given this expression, we can then compute the dimensionality of a dataset using:
$$
d = \frac{\log{n_2} - \log{n_1}}{\log{\epsilon_1} - \log{\epsilon_2}}
$$



Where $(n_1,\epsilon_1)$, $(n_2, \epsilon_2)$ are the number of cells and diameter of each cell at 2 different scales.

### Using K-Means to estimate intrinsic dimension
We can use K-Means to approximate the value of intrinsic dimension of a data-set. In this case, the K centers represent the cells, each with a diameter equal to the Mean Squared Distance of the entire dataset to their representative centers. The estimate for intrinsic dimension then becomes:
$$
d = \frac{\log{n_2} - \log{n_1}}{\log{\sqrt{\epsilon_1}} - \log{\sqrt{\epsilon_2}}} = 2 \times \frac{\log{n_2} - \log{n_1}}{\log{\epsilon_1} - \log{\epsilon_2}}
$$

## Notebook Setup 

In [1]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import *
from math import log
import pickle

In [2]:
spark = SparkSession \
    .builder \
    .getOrCreate()
sc = spark.sparkContext

## Testing on the homework server
On EdX, we provide a test submission and a final submission box. The test submission evaluates your code for the test dataset provided with the homework. The final submission is evaluated on a dataset that is much larger.

For submission and for local testing, make sure to read the path of the file you want to operate with from `./hw5-files.txt`. Otherwise your program will receive no points. This step is exactly the same procedure as your HW2 (Twitter).

## Local test

For local testing, please create your own `hw5-files.txt` file, which contains a single file path on the local disk, e.g.
`file://<absolute_path_to_current_directory>/hw5-small.parquet`. For final submission, we will create this file on our server for testing with the appropriate file path. If your implementation is correct, you should not worry about which file system (i.e. local file system or HDFS) Spark will read data from.

In [3]:
with open('./hw5-files.txt') as f:
    file_path =  f.read().strip()

## Exercises

### Exercise 1: runKmeans

#### Example
The function <font color="blue">runKmeans</font> takes as input the complete dataset rdd, the sample of the rdd on which k-means needs to be run on, the k value and the count of elements in the complete data-set. It outputs the Mean Squared Distance(MSD) of all the points in the dataset from their closest centers, where the centers are calculated using k-means.

**<font color="magenta" size=2>Example Code</font>**
``` python
rdd = sc.parallelize([(1,1),(2,2),(3,3),(4,4),(5,5)])
runKmeans(rdd, sc.parallelize([(1,1),(2,2),(3,3)]), 3, rdd.count())
```

**<font color="blue" size=2>Example Output</font>**
``` python
2.0
```

<font color="red">**Hint : **</font> You might find [K-Means API](https://spark.apache.org/docs/2.2.0/mllib-clustering.html#k-means) useful. Ensure that the initializationMode parameter is set to kmeans++. The computeCost function gives you the sum of squared distances. You might want to tweak maxIterations to compute centers faster

In [9]:
def runKmeans(data, sample_dataset, k, count):
    ### BEGIN SOLUTION
    clusters = KMeans.train(sample_dataset, k, maxIterations=1)
    avg_dists = []
    for i in range(1):
        avg_dists.append(clusters.computeCost(data.map(tuple))/count)
    return sum(avg_dists)
    ### END SOLUTION

### Exercise 2: computeIntrinsicDimension

#### Example
The function <font color="blue">computeIntrinsicDimension</font> takes as input a pair of values $(n1, e1)$, $(n2, e2)$ and computes the intrinsic dimension as output. $e1, e2$ are the mean squared distances of data-points from their closest center

**<font color="magenta" size=2>Example Code</font>**
``` python
n1 = 10
n2 = 100
e1 = 10000
e2 = 100
computeIntrinsicDimension(n1, e1, n2, e2)
```

**<font color="blue" size=2>Example Output</font>**
``` python
1.0
```
<font color="red">**Hint : **</font> Use the last formula in the theory section 

In [5]:
def computeIntrinsicDimension(n1, e1, n2, e2):
    ### BEGIN SOLUTION
    return 2*(log(n2)-log(n1))/(log(e1)-log(e2))
    ### END SOLUTION

### Exercise 3: Putting it all together

#### Example
Now we run K-means for various values of k and use these to estimate the intrinsic dimension of the dataset. Since the dataset might be very large, running kmeans on the entire dataset to find k representative centers may take a very long time. To overcome this, we sample a subset of points from the complete dataset and run Kmeans only on these subsets. We will run Kmeans on 2 different subset sizes: 10000, 20000 points. We will be estimating the MSD for K values of 10, 200, 700, 2000.


The function <font color="blue">run</font> takes a dataframe containing the complete data-set as input and needs to do the following:
* For each sample size S
 * Take the first S elements from the dataframe
 * For each value of K (number of centroids)
  * Call runKmeans(data,S,K,data_count)
* Use the MSD values generated to calculate the intrinsic dimension where $(n_1, n_2) \in \{(10,200),(200,700),(700,2000)\}$.  

**NOTE: Ensure you the format of your output is identical to the one given below, i.e. the keys have to be of the format:**
```python
ID_<Subset_size>_<K-Value-1>_<K-Value-2>
```

**<font color="magenta" size=2>Example Code</font>**
``` python

df = spark.read.parquet(file_path)
run(df)
```

**<font color="blue" size=2>Example Output</font>**
``` python
{'ID_10000_10_200': 1.5574966096390015, 'ID_10000_200_700': 1.3064513902343675, 'ID_10000_700_2000': 1.091310378488035, 'ID_20000_10_200': 1.518279780870003, 'ID_20000_200_700': 1.2660237819996782, 'ID_20000_700_2000': 1.0151131917703071}
```
**Note: The output here is the output of the below function, i.e., the value stored in the variable where the 'run' function is called**

In [6]:
def run(df):
    ### BEGIN SOLUTION
    sample_sizes = [10000, 20000]
    kvalues = [10, 200, 700, 2000]
    count = df.rdd.count()
    res = {}
    msd_dict = {10000: [], 20000:[]}
    for sample_size in sample_sizes:
        samples = df.limit(sample_size)
        for k in kvalues:
            msd_dict[sample_size].append(runKmeans(df.rdd.map(tuple).cache(), samples.rdd.map(tuple).cache(), k, count))
        
    for size in sample_sizes:
        for i in range(len(kvalues)-1):
            res["ID_" + str(size) + "_" + str(kvalues[i]) + "_" + str(kvalues[i+1])] = computeIntrinsicDimension(kvalues[i], msd_dict[size][i], kvalues[i+1], msd_dict[size][i+1])
    return res
    ### END SOLUTION

In [7]:
df = spark.read.parquet(file_path)
res = run(df)
with open('results.pkl', 'wb') as output:
    pickle.dump(res, output, 2, fix_imports=True)

{10000: [0.055228040990767065], 20000: []}
{10000: [0.055228040990767065, 0.001178939396982304], 20000: []}
{10000: [0.055228040990767065, 0.001178939396982304, 0.00017321960695268091], 20000: []}
{10000: [0.055228040990767065, 0.001178939396982304, 0.00017321960695268091, 2.5294678155051414e-05], 20000: []}
{10000: [0.055228040990767065, 0.001178939396982304, 0.00017321960695268091, 2.5294678155051414e-05], 20000: [0.06090628329037355]}
{10000: [0.055228040990767065, 0.001178939396982304, 0.00017321960695268091, 2.5294678155051414e-05], 20000: [0.06090628329037355, 0.0011771747930369803]}
{10000: [0.055228040990767065, 0.001178939396982304, 0.00017321960695268091, 2.5294678155051414e-05], 20000: [0.06090628329037355, 0.0011771747930369803, 0.0001626859023616784]}
{10000: [0.055228040990767065, 0.001178939396982304, 0.00017321960695268091, 2.5294678155051414e-05], 20000: [0.06090628329037355, 0.0011771747930369803, 0.0001626859023616784, 2.0561841483617706e-05]}
